In [1]:
%load_ext sql
%sql sqlite:///../data/publications.db
%config SqlMagic.displaylimit = None

Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'sqlite:///../data/publications.db'

displaylimit: Value None will be treated as 0 (no limit)

# Aufgabe 1 (Komplexere Abfragen)

Realisieren Sie folgende Aufgaben in `publications.db`. (Schema auf einen früheren Übungsblatt).

1.  Bestimmen Sie für jeden Laden und jeden Autor den Gesamtpreis der von diesem Autor in diesem Laden verkauften Bücher (ohne Discounts), absteigend sortiert.

In [89]:
%%sql

with total_quantity_per_store_and_title as ( 
    SELECT sd.stor_id, sd.title_id, sum(sd.qty) as total_quantity
    from salesdetail sd
    group by sd.stor_id, sd.title_id)

SELECT au.au_fname, au.au_lname, st.stor_name, ROUND(sum(t.price * tq.total_quantity),2) as revenue
from authors au
join titleauthor ta on au.au_id = ta.au_id
join titles t on ta.title_id = t.title_id
join total_quantity_per_store_and_title tq on t.title_id = tq.title_id
join stores st on st.stor_id = tq.stor_id
group by au.au_id, st.stor_id
order by au.au_id, revenue desc


Running query in 'sqlite:///../data/publications.db'

au_fname,au_lname,stor_name,revenue
Johnson,White,Thoreau Reading Discount Chain,68705.63
Johnson,White,News & Brews,6896.55
Johnson,White,Eric the Read Books,3998.0
Johnson,White,Bookbeat,1799.1
Marjorie,Green,Thoreau Reading Discount Chain,106896.0
Marjorie,Green,Doc-U-Mat: Quality Laundry and Books,10999.2
Marjorie,Green,Bookbeat,8414.26
Marjorie,Green,Eric the Read Books,7398.45
Marjorie,Green,News & Brews,3406.34
Marjorie,Green,Barnum's,598.0


2.  Bestimmen Sie alle Autoren, die weniger `psychology` Bücher herausgegeben haben als `Livia Karsen`.

In [54]:
%%sql

select * from authors
where au_id not in (
  select distinct a.au_id from authors as a
    natural join titleauthor as ta
    natural join titles as t
  group by a.au_lname, a.au_fname
  having count(*) >= (
    select count(*) as published_psy_books from authors as a
      natural join titleauthor as ta
      natural join titles as t
    where t.type like 'psychology%'
    and a.au_lname = 'Karsen' and a.au_fname = 'Livia'
  )
);

Running query in 'sqlite:///../data/publications.db'

au_id,au_lname,au_fname,phone,address,city,state,country,postalcode
893-72-1158,McBadden,Heather,707 448-4982,301 Putnam,Vacaville,CA,USA,95688
724-08-9931,Stringer,Dirk,415 843-2991,5420 Telegraph Av.,Oakland,CA,USA,94609
527-72-3246,Greene,Morningstar,615 297-2723,22 Graybar House Rd.,Nashville,TN,USA,37215
341-22-1782,Smith,Meander,913 843-0462,10 Mississippi Dr.,Lawrence,KS,USA,66044


3.  Geben Sie für jedes Buch seinen Titel, Preis, Anzahl bisher verkaufter Exemplare sowie die effektiv geschuldeten Tantiemen an. (Die pro Buch geschuldete Tantieme in Prozent des Verkaufspreises ist in der Spalte `royalty` in Abhängigkeit von den Verkaufszahlen angegeben) 

4.  Testen Sie, ob die Werte des Attributs `total_sales` korrekt sind, indem Sie alle Buchtitel mit falschen Werte zusammen mit den Werten asugeben.

5.  Geben Sie die Läden an, die schon Bücher aller Typen verkauft haben.